In [1]:
import numpy as np
from numpy import linalg as la
np.set_printoptions(suppress = True)

**Parametri kamere**

In [2]:
def parametriKamere(T):
    
    (i, j) = np.shape(T)
    if i != 3 or j != 4:
        print("Matrica kamere mora biti dimenzije 3x4")
        return
    
    T0 = T[:, :-1]
    
    [Q, R] = la.qr(la.inv(T0))
    
    for i in range(3):
        if R[i, i] < 0:
            R[i, :] = -R[i, :]
            Q[:, i] = -Q[:, i]
    
    
    K = la.inv(R)
    K = K/K[2][2]
    A = la.inv(Q)
    
    C = T[:, -1]
    C = -la.inv(T0).dot(C)
    
    return K, A, C
    
    

In [3]:
n = int(input("Unesite vrednost n: "))
T = np.array([[5,-1-2*n, 3, 18-3*n],
                 [0,-1,5,21],
                 [0,-1,0,1]
                ])

Unesite vrednost n: 9


In [4]:
[K, A, C] = parametriKamere(T)
print(K, '\n\n', A, '\n\n', C)

[[ 5.  3. 19.]
 [ 0.  5.  1.]
 [ 0.  0.  1.]] 

 [[ 1.  0.  0.]
 [ 0.  0.  1.]
 [-0. -1. -0.]] 

 [ 8.  1. -4.]


**CameraDLP**

In [5]:
def cameraEquations(o, p):
    m = np.matrix([[0, 0, 0, 0, -p[2]*o[0], -p[2]*o[1], -p[2]*o[2],  -p[2]*o[3], p[1]*o[0], p[1]*o[1], p[1]*o[2],p[1]*o[3]],
                   [p[2]*o[0], p[2]*o[1], p[2]*o[2], p[2]*o[3], 0, 0, 0, 0, -p[0]*o[0], -p[0]*o[1], -p[0]*o[2], -p[0]*o[3]]])
    return m

In [6]:
def cameraDLP(originali, projekcije):
    
    matrica = cameraEquations(originali[0], projekcije[0])
    
    for i in range(1, 6):
        m = cameraEquations(originali[i], projekcije[i])
        matrica = np.concatenate((matrica, m), axis=0)
            
    U, D, Vt = la.svd(matrica)
    
    #matrica P ce biti poslednja kolona matrice V sto je ustvari poslednja vrsta matrica Vt
    #skaliramo tako da element na poziciji (0,0) bude 1
    P = Vt[-1] / Vt[-1, 0]
    P = P.round(4)
    P = P.reshape(3, 4)
   
    return P

In [7]:
n = int(input("Unesite vrednost n: "))

M1 = np.array([460, 280, 250, 1])
M2 = np.array([50, 380, 350, 1])
M3 = np.array([470, 500, 100, 1])
M4 = np.array([380, 630, 50 * n, 1])
M5 = np.array([30 * n, 290, 0, 1])
M6 = np.array([580, 0, 130, 1])
originali = np.array([M1, M2, M3, M4, M5, M6])

M1p = np.array([288, 251, 1])
M2p = np.array([79, 510, 1])
M3p = np.array([470, 440, 1])
M4p = np.array([520, 590, 1])
M5p = np.array([365, 388, 1])
M6p = np.array([365, 20, 1])
projekcije = np.array([M1p, M2p, M3p, M4p, M5p, M6p])

P = cameraDLP(originali, projekcije)
print(P)

Unesite vrednost n: 9
[[    1.        -4.0266    11.446  -2932.2824]
 [    5.9927    -3.6374    10.5579 -4908.2022]
 [    0.0108    -0.0053     0.0248   -11.9482]]
